In [1]:
# Imports organizados por funcionalidade
import torch
from torch import nn
import kornia
import itertools
from teste_util import read_dataload_flower, set_seed
from custom_local_feature import REKDSosNet, SingularPointSosNet
from external.hardnet_pytorch import HardNet

# Configurações iniciais
set_seed(42)  # Fixar a semente para reprodutibilidade
# Leitura dos dados
data_dir = './data/datasets'
batch_size = 60
train_loader, test_loader = read_dataload_flower(120, data_dir, batch_size=batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Configuração do detector de pontos-chave
keypoint_detector_config = {
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    'Detector_conf': {
        'nms_size': 5,
        'pyramid_levels': 0,
        'up_levels': 0,
        'scale_factor_levels': 1.0,
        's_mult': 5.0,
    },
}

singular_point_detector = (
    SingularPointSosNet(config=keypoint_detector_config, device=device)
    ._initialize_detector(num_features=60, size_laf=32)
    .to(device)
)

# Classe para o descritor HardNet
class HardNetDescriptor(nn.Module):
    def __init__(self):
        super().__init__()
        hardnet = HardNet()
        checkpoint_path = 'trained_models/pretrained_nets/HardNet++.pth'
        checkpoint = torch.load(checkpoint_path)
        hardnet.load_state_dict(checkpoint['state_dict'])
        hardnet.eval()
        hardnet.to(device)
        self.model = hardnet

    def forward(self, x):
        return self.model(x)

# Inicialização dos descritores
hardnet_descriptor = HardNetDescriptor()
sift_descriptor = kornia.feature.SIFTDescriptor(32, rootsift=True)

# Dicionário com detectores e descritores já instanciados
detectors = {
    "singular_point": singular_point_detector,  # Instância do detector
}

descriptors = {
    "hardnet": hardnet_descriptor,  # Instância do descritor HardNet
    "sift": sift_descriptor,        # Instância do descritor SIFT
}

thresholds = {
    "alpha_threshold": [0.5, 1.0, 1.5],  # Lista de thresholds
}

def generate_combinations(detectors, descriptors, thresholds):
    # Produto cartesiano entre detectores, descritores e thresholds
    return list(itertools.product(detectors.items(), descriptors.items(), thresholds["alpha_threshold"]))

# Gerar as combinações
combinations = generate_combinations(detectors, descriptors, thresholds)


6072


/home/wagner/anaconda3/envs/singular-points/lib/python3.9/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)
/home/wagner/Documentos/python/singular-points2/teste_util.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed 

Model loaded from ./data/models/sp_map_fo_30.pth
Model loaded from ./data/models/sp_map_fo_30.pth


/tmp/ipykernel_743046/2773550758.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [2]:
from kornia.feature import laf_from_center_scale_ori
def convert_points_to_lafs(points,img1, PS=19,scale=6):
    orient = kornia.feature.LAFOrienter(PS)#kornia.feature.LAFOrienter(PS)PassLAF()
    scale_lafs = torch.ones(img1.shape[0],points.shape[1],1,1)*scale
    scale_lafs = scale_lafs.to(img1.device)
    points = points.to(img1.device)
    lafs1 = laf_from_center_scale_ori(points,scale_lafs)
    lafs2 = orient(lafs1, img1)
    return lafs2
    
def extract_patches_simple(batch, lafs, PS=19):
    imgs_patches = kornia.feature.extract_patches_from_pyramid(batch, lafs, PS)
    imgs_patches =imgs_patches.reshape(-1,imgs_patches.shape[2],PS,PS)
    return imgs_patches

def plot_patches_side_by_side(imgs_patches):
    num_imgs = imgs_patches.shape[0]  # Número de imagens
    fig, axs = plt.subplots(1, num_imgs, figsize=(num_imgs*4, 4))

    axs = axs.reshape((1, num_imgs))  # Ajustar a forma para matriz 2D com uma única linha

    for i in range(num_imgs):
        axs[0, i].imshow(kornia.tensor_to_image(imgs_patches[i]))
        axs[0, i].axis('off')

    plt.show()

In [3]:
import numpy as np
from scipy.spatial.distance import cdist
import kornia.feature as KF
import matplotlib.pyplot as plt
import kornia as K

def visualize_LAF(img, LAF, img_idx = 0):
    x, y = KF.laf.get_laf_pts_to_draw(LAF, img_idx)
    print(x[0][:5],y[0][:5])
    plt.figure()
    plt.imshow(K.utils.tensor_to_image(img[img_idx]))
    plt.plot(x, y, 'r')
    plt.show()
    return

def plot_matches_keypoints(image1, keypoints1, image2, keypoints2, matches, **kwargs):
    print('image1 shape: ',image1.shape,image1.dtype,image2.shape,image2.dtype)
    # Concatenar as duas imagens lado a lado
    combined_image = np.concatenate((image1, image2), axis=1)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(combined_image)
    ax.axis('off')

    # Desenhar pontos correspondentes e linhas conectando-os
    offset = image1.shape[1]

    for i, (x, y) in enumerate(keypoints1):
        ax.plot(x, y, 'o',markerfacecolor='none', markeredgecolor='r',
                markersize=20, markeredgewidth=1)
        ax.annotate(str(i), (x, y), color='r',xytext=(10, 10), textcoords='offset points', fontsize=12)

    for i, (x, y) in enumerate(keypoints2):
        ax.plot(x+offset, y, 'o',markerfacecolor='none', markeredgecolor='r',
                markersize=20, markeredgewidth=1)
        ax.annotate(str(i), (x+offset, y), color='r',xytext=(10, 10), textcoords='offset points', fontsize=12)

    for match in matches:
        x1, y1 = keypoints1[match[0],0], keypoints1[match[0],1]
        x2, y2 = keypoints2[match[1],0]+offset, keypoints2[match[1],1]
        ax.plot([x1, x2], [y1, y2], '-', color='lime', lw=0.5)

    plt.tight_layout()
    plt.show()

def plot_image_with_keypoints(image_tensor, keypoints_tensor):
    # Converter a imagem tensorial em objeto PIL.Image
    image = kornia.utils.tensor_to_image(image_tensor)
    # Plotar a imagem e os keypoints
    plt.imshow(image)
    if keypoints_tensor is not None:
        # Extrair as coordenadas x e y dos keypoints
        keypoints_x = keypoints_tensor[:,0].flatten().tolist()
        keypoints_y = keypoints_tensor[:,1].flatten().tolist()
        plt.scatter(keypoints_x, keypoints_y, c='red')
    plt.show()
    
def filtrar_keypoints(lista_de_pontos, tensor_mascara):
    # Verificar se as coordenadas estão dentro das dimensões
    dimensao_max_x, dimensao_max_y = tensor_mascara.shape[1] - 1, tensor_mascara.shape[0] - 1
    pontos_filtrados = [
        ponto.tolist()  for ponto in lista_de_pontos 
        if 0 <= ponto[0] <= dimensao_max_x 
        and 0 <= ponto[1] <= dimensao_max_y 
        and tensor_mascara[int(ponto[1]), int(ponto[0])] 
    ]
    return torch.tensor(pontos_filtrados)

def filtrar_keypoints_conjuntos(lista_de_pontos_1, lista_de_pontos_2, tensor_mascara):
    dimensao_max_x, dimensao_max_y = tensor_mascara.shape[1] - 1, tensor_mascara.shape[0] - 1
    pontos_filtrados_1 = []
    pontos_filtrados_2 = []
    
    for ponto_1, ponto_2 in zip(lista_de_pontos_1.cpu(), lista_de_pontos_2.cpu()):
        x, y = ponto_1
        if 0 <= x <= dimensao_max_x and 0 <= y <= dimensao_max_y and tensor_mascara[int(y), int(x)]:
            pontos_filtrados_1.append(ponto_1.numpy())
            pontos_filtrados_2.append(ponto_2.numpy())
            
    return torch.tensor(pontos_filtrados_1), torch.tensor(pontos_filtrados_2)

def find_best_matching_indices_knn(points1, points2, threshold, k=3):
    if len(points1) == 0 or len(points2) == 0:
        return []
    distances = cdist(points1, points2)
    best_indices = np.argsort(distances, axis=1)[:, :k]
    best_distances = np.take_along_axis(distances, best_indices, axis=1)

    matched = []

    for i in range(len(points1)):
        min_distance = np.min(best_distances[i])
        if min_distance < threshold:
            best_index = np.argmin(best_distances[i])
            matched.append((i, best_indices[i, best_index]))

    return matched

def calcular_associacao(matches1, matches2):
    intersecao = set(tuple(match) for match in matches1) & set(tuple(match) for match in matches2)
    # assoc_score = len(intersecao) / min(matches1.shape[0], matches2.shape[0])
    return intersecao #assoc_score


def detect_extract_feat_in_batch(batch1,aug_list, detector,descritor,alpha_threshold):
    total = []
    intersecao_total = []
    with torch.no_grad():
        for img1  in batch1:            
            lafs1, resps1 = detector(img1[None])
            B,C,H,W = img1[None].shape
            mask = torch.ones(B,C,H,W).to(img1.device) 
            #lafs1 to points1
            points1 =kornia.feature.get_laf_center(lafs1)   

            if( points1.shape[1] == 0):
                # print('aug_list shape: ',points1.shape) 
                continue                    
            params = next(aug_list)    
            img2,mask_t,ponts_t=aug_list.augmentation_sequence(img1,mask,points1,params=params)    
            img2 = img2.to(img1.device)                   
            lafs2, resps2 = detector(img2)            
            points2 =kornia.feature.get_laf_center(lafs2)
            # visualize_LAF(img2, lafs2, 0)
                        
            # pontos filtrados com base da mascara
            filtered_points1,filtered_points0 = filtrar_keypoints_conjuntos(ponts_t[0],points1[0],mask_t[0,0].bool())
            filtered_points2 = filtrar_keypoints(points2[0],mask_t[0,0].bool())                        
            # print('filtered shape: ',filtered_points1.shape,filtered_points2.shape)
            if( filtered_points1.shape[0] == 0 or filtered_points2.shape[0] == 0):
                # print('filtered shape: ',filtered_points1.shape,filtered_points2.shape)
                continue                        
            
            matches = find_best_matching_indices_knn(filtered_points1.cpu(), filtered_points2.cpu(), threshold=alpha_threshold, k=1)
            if(len(matches) == 0):
                # print('matches shape: ',len(matches))
                continue
            
            # print('filtered_points1 shape: ',filtered_points0.shape,filtered_points1.shape,filtered_points2.shape)
            lafs1 = convert_points_to_lafs(filtered_points0[None],img1[None], PS=19,scale=5)
            lafs2 = convert_points_to_lafs(filtered_points2[None],img2, PS=19,scale=5)
            # print('lafs1 shape: ',lafs1.shape,lafs2.shape,img1.shape,img2.shape)
            patch1 = extract_patches_simple(img1[None], lafs1, PS=32)# TODO 13 para sift e 32 hardnet
            patch2 = extract_patches_simple(img2, lafs2, PS=32)# TODO 13 para sift e 32 hardnet
 
            B, N, CH, H, W = patch1[None].size()       
            # print(B, N, CH, H, W) 
            desc1 =descritor(patch1.view(B * N, CH, H, W))
            B, N, CH, H, W = patch2[None].size()
            desc2 =descritor(patch2.view(B * N, CH, H, W))                        
            #TODO: verificar a correspondencia entre os descritores

            dist,match_desc = kornia.feature.match_smnn(desc1, desc2, th=0.8) 
            
            # print('calcular_associacao ',match_desc.shape,len(matches))            
            intersecao = calcular_associacao(match_desc.cpu().numpy(), np.array(matches)) 
            total.append(len(matches))
            intersecao_total.append(len(intersecao))            
            # plot_matches_keypoints(img2[0,0].cpu().numpy(), filtered_points1.cpu().numpy(), img2[0,0].cpu().numpy(), filtered_points2.cpu().numpy(), matches)
            # plot_matches_keypoints(img2[0,0].cpu().numpy(), filtered_points1.cpu().numpy(), img2[0,0].cpu().numpy(), filtered_points2.cpu().numpy(), match_desc)
    # print('total: ',np.sum(total),' intersecao: ',np.sum(intersecao_total))
    return (np.sum(intersecao_total)/np.sum(total))*100



In [4]:
class AugmentationGenerator:
    def __init__(self, n_variations):
        # Definir as augmentações
        aug_gen = kornia.augmentation.AugmentationSequential(
            kornia.augmentation.RandomAffine(degrees=360, translate=(0.2, 0.2), scale=(0.95, 1.05), shear=10,p=0.8),
            kornia.augmentation.RandomPerspective(0.2, p=0.7),
            kornia.augmentation.RandomBoxBlur((4,4),p=0.5),
            data_keys=[kornia.constants.DataKey.INPUT,  # Especifica as chaves corretamente
                       kornia.constants.DataKey.MASK,
                       kornia.constants.DataKey.KEYPOINTS],
            same_on_batch=True,
        )

        self.augmentation_sequence = aug_gen
        self.n_variations = n_variations
        self.param_list = []
        self.current_index = 0

    def generate_variations(self, image, mask, keypoints):
        """
        Gera múltiplas variações de augmentações e coleta seus parâmetros.
        """
        for _ in range(self.n_variations):
            # Apenas executa a sequência de augmentação e salva os parâmetros gerados
            self.augmentation_sequence(image, mask, keypoints)
            self.param_list.append(self.augmentation_sequence._params)

    def __iter__(self):
        self.current_index = 0  # Resetar o índice a cada nova iteração
        return self

    def __next__(self):
        """
        Retorna a próxima variação de parâmetros de augmentação.
        A iteração será circular.
        """
        result = self.param_list[self.current_index]
        self.current_index = (self.current_index + 1) % len(self.param_list)
        return result

    def reset(self):
        """Método para resetar o estado do gerador de augmentação."""
        self.current_index = 0  # Reseta o índice de iteração

In [5]:
from tqdm.notebook import tqdm

set_seed(42)
aug_gen = AugmentationGenerator(15)
image = torch.rand(1, 1, 120, 120)  # Imagem com valores aleatórios
mask = torch.ones(1, 1, 120, 120)  # Máscara binária
keypoints = torch.tensor([[[30, 30], [90, 90]]], dtype=torch.float32)  # Pontos chave de exemplo

# Gerar as variações
aug_gen.generate_variations(image, mask, keypoints)
# Loop pelas combinações geradas
for (detector_name, detector), (descriptor_name, descriptor), alpha_threshold in combinations:
    # Log dos parâmetros atuais
    pbar = tqdm(test_loader, desc=f"Evaluation {detector_name} - {descriptor_name} - {alpha_threshold}")  # Usando f-string para formatar o nome da classe do detector
    list_acc = []
    for imgs_batch,labels_batch in pbar:# itera em todo dataset
        imgs_batch = imgs_batch.to(device)
        _acc = detect_extract_feat_in_batch(imgs_batch,aug_gen,detector,descriptor,alpha_threshold)
        list_acc.append(_acc)
        pbar.set_postfix({"Acc.": f"{_acc:.4f}", "Mean Acc.": f"{np.mean(list_acc):.4f}"})
    aug_gen.reset()


/home/wagner/anaconda3/envs/singular-points/lib/python3.9/site-packages/torch/nn/functional.py:4969: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/wagner/anaconda3/envs/singular-points/lib/python3.9/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


Evaluation singular_point - hardnet - 0.5:   0%|          | 0/102 [00:00<?, ?it/s]

/tmp/ipykernel_743046/2450327458.py:80: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(pontos_filtrados_1), torch.tensor(pontos_filtrados_2)


Evaluation singular_point - hardnet - 1.0:   0%|          | 0/102 [00:00<?, ?it/s]

Evaluation singular_point - hardnet - 1.5:   0%|          | 0/102 [00:00<?, ?it/s]

Evaluation singular_point - sift - 0.5:   0%|          | 0/102 [00:00<?, ?it/s]

Evaluation singular_point - sift - 1.0:   0%|          | 0/102 [00:00<?, ?it/s]

Evaluation singular_point - sift - 1.5:   0%|          | 0/102 [00:00<?, ?it/s]

Evaluation singular_point - hardnet - 0.5: 100%  102/102 [09:14<00:00,  4.22s/it, Acc.=65.4762, Mean Acc.=68.5757]
Evaluation singular_point - hardnet - 1.0: 100%  102/102 [09:09<00:00,  3.93s/it, Acc.=53.2520, Mean Acc.=58.6233]
Evaluation singular_point - hardnet - 1.5: 100%  102/102 [08:50<00:00,  4.02s/it, Acc.=45.7055, Mean Acc.=51.0960]

Evaluation singular_point - sift - 0.5: 100%  102/102 [08:57<00:00,  3.98s/it, Acc.=59.5238, Mean Acc.=66.1388]
Evaluation singular_point - sift - 1.0: 100%  102/102 [08:58<00:00,  4.03s/it, Acc.=47.9675, Mean Acc.=55.4892]
Evaluation singular_point - sift - 1.5: 100%  102/102 [08:59<00:00,  4.07s/it, Acc.=41.1043, Mean Acc.=47.7420]

---------------------------------------------------------------
kornia-matching-test.

ours	sift	0.5 match of dataset  68.12792897842729
ours	sift	1.0 match of dataset  57.2651854514897
ours	sift	1.5 match of dataset  48.07975034660492

ours	hardnet	0.5 match of dataset  69.75870075937866
ours	hardnet	1.0 match of dataset  59.41000414346953
ours	hardnet	1.5 match of dataset  50.44833692096903

match of dataset  47.742045293607646